# Extraction Script for VTP Sketch Model
## To pull results from base run of GTAM into VTP Sketch Model
Peter - 20220316

In [ ]:
import os
import glob
import inro.modeller as m
import inro.emme.matrix as _matrix
import numpy as np
import pandas as pd
mm = m.Modeller()    
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
eb = mm.emmebank

sov_dist = 'mf180'
truck_cost = 'mf190'
active_time = 'mf170'
active_dem_offpk = 'mf160'
active_dem_pk = 'mf150'

In [ ]:
## Run Folder
runpath = r'\\tore-infs01\Models\VaughanTMP\2051_ForecastRuns\2051_Preferred_new_20211209_1'
scen = 10003

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"


def create_directory_dict(directory, file_ext):
    direc = directory # Get current working directory
    ext = file_ext # Select your file delimiter
    file_dict = {} # Create an empty dict
    # Select only files with the ext extension
    txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]
    
    # Iterate over your txt files
    for f in txt_files:
        file_dict[f] = ""
        
    oldkeys = list(file_dict.keys())
    newkeys = [s.replace('.mtx', '') for s in oldkeys]
    vals = list(file_dict.values())
    new_dict = {k: v for k, v in zip(newkeys, vals)}
    return new_dict

def Merge(dict1, dict2):
    return(dict2.update(dict1))

Import demand matrices

In [ ]:
demand_folder = os.path.join(runpath, 'Demand')
mat_list = create_directory_dict(demand_folder, '.mtx')

for matrix_name in mat_list.keys():
    mat_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

import_matrices_from_directory(demand_folder, mat_list, ".mtx", scen)

Import LOS Skims

In [ ]:
LOS_folder = os.path.join(runpath, 'LOS Matrices/AM')
mat_list_2 = create_directory_dict(demand_folder, '.mtx')

for matrix_name in mat_list_2.keys():
    mat_list_2[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list_2[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

import_matrices_from_directory(LOS_folder, mat_list_2, ".mtx", scen)

matrix_list = Merge(mat_list, mat_list_2)

Calculate Transit Access Time

In [ ]:
transit_access_time = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = transit_access_time,
              matrix_name="transit.access.time",
              matrix_description = "transit.access.time")

exp = matrix_list[twait] + "+" + matrix_list[twalk]

spec = {
    "type": "MATRIX_CALCULATION",
    "result": transit_access_time,
    "expression": exp
}
matrix_calculator(spec)

Output Columns

In [ ]:
### Peak period is AM, Offpeak period is MD
output_cols = {
    "sov_dist":sov_dist,
    "sov_time":matrix_list['aivtt'],
    "ff_time":matrix_list['MD_aivtt'],
    "active_time": active_time, 
    "sov_cost":matrix_list['acost'], 
    "transit_cost": matrix_list['tfare'],
    "light_truck_pkdem":matrix_list['AMLightMatrix'],
    "med_truck_pkdem":matrix_list['AMMediumMatrix'],
    "heavy_truck_pkdem":matrix_list['AMHeavyMatrix'],
    "light_truck_offpkdem":matrix_list['MDLightMatrix'],
    "med_truck_offpkdem":matrix_list['MDMediumMatrix'],
    "heavy_truck_offpkdem":matrix_list['MDHeavyMatrix'],
    "transit_access_time": transit_access_time,
    "transit_time": matrix_list['tivtt'],
    "active_dem_pk":active_dem_pk,
    "active_dem_offpk":active_dem_offpk,
    "sov_dem_pk":matrix_list['AMAutoMatrix'],
    "sov_dem_offpk":matrix_list['MDAutoMatrix'],
    "transit_dem_pk":matrix_list['AMTransitMatrix'],
    "transit_dem_offpk":matrix_list['MDTransitMatrix'],
    "truck_cost": truck_cost
}

# posted speed to 60, set 

first = True
indices = eb.matrix(matrix_list[sov_demand_pk]).get_data(scen).indices
for col in output_cols:
    mat_np = eb.matrix(output_cols[col]).get_numpy_data(scen)
    if first:
        df_full = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: col})
        first = False
    else:
        df = pd.DataFrame(mat_np,indices[0],indices[1]).stack().reset_index().rename(columns = {"level_0": "O",
                                                                                           "level_1": "D",
                                                                                          0: col})
        df_full = df_full.merge(df,on = ["O","D"])

In [ ]:
df_full.columns

In [ ]:
df_full.sov_cost = df_full.sov_cost/100
df_full.hov_cost = df_full.hov_cost/100
df_full["dem_truck_pk"] = df_full.light_truck_pkdem + df_full.med_truck_pkdem + df_full.heavy_truck_pkdem
df_full["dem_truck_offpk"] = df_full.light_truck_offpkdem + df_full.med_truck_offpkdem + df_full.heavy_truck_offpkdem 
df_full["dem_pk"] = df_full.sov_dem_pk + df_full.hov_dem_pk+df_full.transit_dem_pk+df_full.active_dem_pk

In [ ]:
# demand_weighted = {u'active_time':u'active_dem_pk',u'ff_time':u'sov_dem_pk',u'sov_time':u'sov_dem_pk',
#                    u'hov_cost':u'hov_dem_pk',u'sov_dist':u'sov_dem_pk',u'hov_time':u'sov_dem_pk',
#                    u'sov_cost':u'sov_dem_pk'}

demand_weighted = {
    u'active_time':"dem_pk",u'ff_time':"dem_pk",u'sov_time':"dem_pk",
    u'hov_cost':"dem_pk",u'sov_dist':"dem_pk",u'hov_time':"dem_pk",
    u'sov_cost':"dem_pk",u'transit_cost':"dem_pk",u'transit_time':"dem_pk",
    "truck_cost":"dem_truck_pk","transit_access_time":"dem_pk"}

In [ ]:
for met in demand_weighted.keys():
    df_full[met + "_dem_weight"] = df_full[met]*df_full[demand_weighted[met]]

In [ ]:
pth = r"C:\Users\pechen\Desktop\VTP\SkM"
megazones_df = pd.read_csv(os.path.join(pth,"TAZ_SketchModelZones_20201110.csv"))

In [ ]:
df_internal = df_full[(df_full.O <= 6000) & (df_full.D <= 6000)]

In [ ]:
df_internal = df_internal.merge(megazones_df, left_on = "O", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_O"})
df_internal = df_internal.merge(megazones_df, left_on = "D", right_on = "TAZ").drop(labels = "TAZ",axis = 1).rename(columns = {"Megazone": "Megazone_D"})

In [ ]:
df_internal.columns

In [ ]:
def average(list):
    return sum(list)/len(list)

In [ ]:
def median(list):
    return (max(list) + min(list))/2

In [ ]:
pivot_columns = [u'active_time_dem_weight',u'ff_time_dem_weight',u'sov_time_dem_weight',u'sov_cost_dem_weight',
                u'hov_cost_dem_weight',u'sov_dist_dem_weight',u'hov_time_dem_weight',
                u'active_dem_offpk', u'transit_dem_pk', u'active_dem_pk', u'transit_dem_offpk',
                u'hov_dem_offpk', u'sov_dem_offpk', u'dem_truck_pk', u'dem_truck_offpk', u'hov_dem_pk', u'sov_dem_pk',"dem_pk",
                u'transit_cost_dem_weight',u'transit_time_dem_weight',"truck_cost_dem_weight","transit_access_time_dem_weight"]
megazone_df = pd.pivot_table(df_internal,values = pivot_columns, index = ["Megazone_O","Megazone_D"],aggfunc = sum).reset_index()

In [ ]:
for met in demand_weighted.keys():
    megazone_df[met] = megazone_df[met + "_dem_weight"] / megazone_df[demand_weighted[met]]
    megazone_df = megazone_df.drop(labels = met + "_dem_weight", axis = 1)

In [ ]:
megazone_df.fillna(0).to_csv(os.path.join(pth, "2051N1LUV3_MegazoneOutputs_20210225.csv"))

In [ ]:
megazone_df.fillna(0).to_csv(os.path.join(pth, "2051N1LUV3_MegazoneOutputs_20210225.csv"))